(datospp)=
# Datos pre-procesados
En la sección anterior, analizamos los datos públicados para las LHCO 2020 sin alteración. En esta sección, analizaremos los datos pre-procesados de estos conjuntos. 

Al igual que en la sección anterior, utilizamos los primeros 100,000 eventos de cada conjunto de datos.

Los datos pre-procesados de los primeros 100,000 eventos se obtienen utilizando `build_features` de `benchtools.src.clustering`, que utiliza la librería `pyjet`. Los detalles del pre-procesamiento se encuentran en la {numref}`bench-pre`.

In [1]:
# Importamos las librerías principales
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from myst_nb import glue
from PIL import Image
import os

# Funciones de benchtools
from benchtools.src.plotools import bkg_sig_hist, bkg_sig_scatter
from benchtools.src.clustering import build_features

In [2]:
# Definimos donde están los datos
path_data_RnD = "../../../datos/events_anomalydetection.h5"
path_data_BB1 = "../../../datos/events_LHCO2020_BlackBox1.h5"
path_key_BB1 = "../../../datos/events_LHCO2020_BlackBox1.masterkey"

In [3]:
# Pre-procesamiento R&D
# Esta celda se corre una vez para pre-procesar los datos
# Una vez que el archivo existe no vuelve a correr
build_features(path_data=path_data_RnD, nbatch=2, outname='RnD-100000', outdir='../../../datos/', chunksize=50000)

A file with that name already exists


In [4]:
# Pre-procesamiento BB1
# Esta celda se corre una vez para pre-procesar los datos
# Una vez que el archivo existe no vuelve a correr
build_features(path_data=path_data_BB1, nbatch=2, outname='BB1-100000', path_label=path_key_BB1, outdir='../../../datos/', chunksize=50000)

A file with that name already exists


In [5]:
# Cargamos los datos
df_RnD = pd.read_csv("../../../datos/RnD-100000.csv")
df_BB1 = pd.read_csv("../../../datos/BB1-100000.csv")

glue("df-RnD", df_RnD.head(), display=False)

```{glue:figure} df-RnD
:figwidth: 700px
:name: "df-RnD"

Primeros 5 eventos de los datos R&D pre-procesados usando `benchtools`.
```
(datospp-dist)=
## Distribuciones
Son de interés las variables que posean diferencias entre eventos de señal y de fondo, puesto que en principio, estas son las variables que los podelos van a utilizar para distinguir entre las clases. 

In [28]:
def image_grid(rows, columns, images, title, remove=True):
    # getting sized
    width = []
    height = []
    for image in images:
        img = Image.open(image)
        w, h = img.size
        width.append(w)
        height.append(h)

    wmax = max(width)
    hmax = max(height)

    new_image = Image.new('RGB', (columns*wmax, rows*hmax), color = 'white')
    if len(images) % 2 != 0:
        img = Image.new("RGB", (wmax, hmax), (255, 255, 255))
        img.save('./../../figuras/whiteimage.png', "PNG")
        images.append('./../../figuras/whiteimage.png')
    
    ii = 0
    for row in range(rows):
        for column in range(columns):
            img = Image.open(images[ii])
            new_image.paste(img, (column*wmax, row*hmax))
            ii+=1

    # save it
    new_image.save('./../../figuras/{}.png'.format(title))

    # Eliminando las imagenes individuales
    if remove is True:
        for img in images: 
            os.remove(img)
        
    return None


def create_png(namedf, df, variables, prefilename, nbins=50, type='distribution'):
    lista= []
    for variable in variables:
        # Graficamos
        fig = plt.figure(facecolor='white') # Fondo blanco para la imagen
        if type == 'distribution':
            bkg_sig_hist(df, variable=variable, label='label', n_bins=nbins)
            plt.title('{}: distribución de {}'.format(namedf, variable))
            # Definimos el path y nombre del archivo
            filename = os.path.join('../../figuras/','{}-{}-{}.png'.format(prefilename,namedf,variable))
        if type == 'scatter':
            bkg_sig_scatter(df, variable[0], variable[1], title='{}: correlación entre {} y {}'.format(namedf,variable[0], variable[1]))
            # Definimos el path y nombre del archivo
            filename = os.path.join('../../figuras/','{}-{}-{}v{}.png'.format(prefilename,namedf,variable[0], variable[1]))
        # Guardamos los nombres de los archivos para hacer grid
        lista.append(filename)
        # Guardamos la figura
        plt.savefig(filename, bbox_inches='tight', facecolor=fig.get_facecolor(),edgecolor='none')
        plt.close()
    return lista

In [20]:
variables = ['pT_j1','E_j1', 'm_j1','tau_21_j1']
lista_RnD = create_png('RnD', df_RnD, variables, 'vardiff', nbins=50, type='distribution')
lista_BB1 = create_png('BB1', df_BB1, variables, 'vardiff', nbins=50, type='distribution')
# Grid de imagenes para RnD y BB1
image_grid(rows=2, columns=2, images=lista_RnD, title='datospp-vardiff-RnD')
image_grid(rows=2, columns=2, images=lista_BB1, title='datospp-vardiff-BB1')

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

```{figure} ./../../figuras/datospp-vardiff-RnD.png
---
name: datospp-vardiff-RnD
width: 70%
---
Distribución de $p_T$, $E$, $m$ y $\tau_{21}$ para el conjunto R&D.
```
Observamos que el $p_T$ del fondo decae suavemente, lo que indica que los jets más energéticos de eventos de QCD tienden a tener un momento transverso menor a 1500 GeV. Los jets más energéticos de los eventos de señal poseen un $p_T$ mayor, entre 1500 y 2000 GeV. Esto también se observa en las distribuciones de energía, donde los jets principales de eventos de fondo son más energéticos que los de eventos de señal

La distribución de masa del jet principal también es diferente. Para el fondo, los jets tienden a ser menos masivos y también decaen suavemente. Sin embargo, en los eventos de señal se observan dos picos alrededor de 100 y 500 GeV. Estas son las masas de las partículas $X$ y $Y$ del evento $Z'\rightarrow XY$ para el conjunt R&D.

Finalmente, se observa una diferencia en la distribución de la variable $\tau_{21}$. De acuerdo a su definición en la ec.{eq}`jets-ratio_subjettiness`, el valor de esta variable es menor cuando los jets son más consistentes con la subestructura de dos subjets. Por lo tanto, como la señal se agrupa hacia valores más pequeños, los jets de señal son más consistentes con la subestructura de dos subjets.

Ambos conjuntos presentan distribuciones similares.

```{figure} ./../../figuras/datospp-vardiff-BB1.png
---
name: datospp-vardiff-BB1
width: 70%
---
Distribución de $p_T$, $E$, $m$ y $\tau_{21}$ para el conjunto BB1.
```
La diferencia principal se encuentra en la distribución de la masa del jet. Para el conjunto BB1 los picos de señal están alrededor de 380 y 750 GeV, cercano al valor de las masas de las partículas $X$ y $Y$ de este conjunto.

Las distribuciones de $\eta$ y el número de hadrones que constituyen el jet principal son más angostas para la señal. El ángulo $\phi$ de los jets no presenta diferencias marcadas. 

En general, el bajo porcentaje de señal en el conjunto BB1 no permite observar con exactitud la distribución de la señal, pero es evidente la semejanza con el conjunto R&D.

In [21]:
variables = ['eta_j1', 'nhadrons_j1', 'phi_j1']
lista_RnD = create_png('RnD', df_RnD, variables, 'vareq', nbins=50, type='distribution')
lista_BB1 = create_png('BB1', df_BB1, variables, 'vareq', nbins=20, type='distribution')

# Grid de imagenes para RnD y BB1
image_grid(rows=1, columns=3, images=lista_RnD, title='datospp-vareq-RnD')
image_grid(rows=1, columns=3, images=lista_BB1, title='datospp-vareq-BB1')

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

```{figure} ./../../figuras/datospp-vareq-RnD.png
---
name: datospp-vareq-RnD
width: 100%
---
Distribución de $\eta$, *nro. de hadrones* y $\phi$ del jet principal para el conjunto R&D.
```

```{figure} ./../../figuras/datospp-vareq-BB1.png
---
name: datospp-vareq-BB1
width: 100%
---
Distribución de $\eta$, *nro. de hadrones* y $\phi$ del jet principal para el conjunto BB1.
```
Las distribuciones de estas variables para el jet secundario poseen las mismas características que las del jet principal.

In [22]:
variables = ['pT_j2','E_j2', 'm_j2','tau_21_j2', 'eta_j2', 'nhadrons_j2', 'phi_j2']
lista_RnD = create_png('RnD', df_RnD, variables, 'jet2', nbins=50, type='distribution')
lista_BB1 = create_png('BB1', df_BB1, variables, 'jet2', nbins=20, type='distribution')

# Grid de imagenes para RnD y BB1
image_grid(rows=2, columns=4, images=lista_RnD, title='datospp-jet2-RnD')
image_grid(rows=2, columns=4, images=lista_BB1, title='datospp-jet2-BB1')

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

...

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

```{figure} ./../../figuras/datospp-jet2-RnD.png
---
name: datospp-jet2-RnD
width: 100%
---
Distribución de las variables del jet secundario para el conjunto R&D.
```

```{figure} ./../../figuras/datospp-jet2-BB1.png
---
name: datospp-jet2-BB1
width: 100%
---
Distribución de las variables del jet secundario para el conjunto BB1.
```
Por último, están las variables que relacionan los dos jets principales, $\Delta R$ y $m_{jj}$. En estas variables se observa diferencia entre las distribuciones de señal y de fondo.

In [33]:
lista_RnD = create_png('RnD', df_RnD, ['deltaR_j12','m_jj'], 'jetrelations', type='distribution')
lista_BB1 = create_png('BB1', df_BB1, ['deltaR_j12','m_jj'], 'jetrelations', type='distribution')

# Grid de imagenes para RnD y BB1
image_grid(rows=1, columns=2, images=lista_RnD, title='datospp-jetrelations-RnD')
image_grid(rows=1, columns=2, images=lista_BB1, title='datospp-jetrelations-BB1')

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

```{figure} ./../../figuras/datospp-jetrelations-RnD.png
---
name: datospp-jetrelations-RnD
width: 80%
---
Distribución de las variables $\Delta R$ y $m_{jj}$ para el conjunto R&D.
```

In [23]:
mjj_RnD = df_RnD.loc[df_RnD['label']==1]['m_jj'].mean()
glue('mjj-RnD', mjj_RnD, display=True)
mjj_BB1  = df_BB1.loc[df_BB1['label']==1]['m_jj'].mean()
glue('mjj-BB1', mjj_BB1, display=True)

3475.979232378178

3672.7057998092414


La distribución de $\Delta R$ es más angosta para los eventos de señal. La masa invariante del fondo se encuentra entre 2000 y 3000 GeV, y decae suavemente, mientras que para la señal del conjunto R&D se observa un pico en {glue:text}`mjj-RnD:.0f` GeV, consistente con la masa del bosón $Z'$ de nueva física para el evento de este conjunto de datos. Para el conjunto BB1 se observa un pico en {glue:text}`mjj-BB1:.0f`, cerca de la masa de el bosón $Z'$ para este conjunto.

```{figure} ./../../figuras/datospp-jetrelations-BB1.png
---
name: datospp-jetrelations-BB1
width: 80%
---
Distribución de las variables $\Delta R$ y $m_{jj}$ para el conjunto BB1.
```
Aunque las variables de masa poseen diferentes distribuciones para señal y fondo, y podrían ser de utilidad para que el modelo aprenda, no serán utilizadas para el entrenamiento ni la clasificación. La idea es que la búsqueda sea lo más libre de modelo posible, por lo que no se desea que los algoritmos aprendan la distribución de masa.
(datospp-correlaciones)=
## Correlaciones
La correlación de las variables que se utilizarán para entrenamiento se pueden observar a continuación.

In [103]:
for label, intlabel in [('señal', 1), ('fondo', 0)]:
    # Queremos tener las correlaciones para la señal y el fondo en dataframes separados
    df_BB1_nm = df_BB1[df_BB1['label']==intlabel].drop(['label','m_jj','m_j1','m_j2'], axis=1)
    df_RnD_nm = df_RnD[df_RnD['label']==intlabel].drop(['label','m_jj','m_j1','m_j2'], axis=1)

    # Calculamos las correlaciones
    df_BB1_corr = df_BB1_nm.corr()
    df_RnD_corr = df_RnD_nm.corr()
    
    # Definimos figura y tamaño
    fig, axes = plt.subplots(figsize=[14,11], facecolor='white', nrows=1, ncols=2, sharey=True)
    fig.tight_layout() 
    # Para graficar el mapa de correlaciones:
    for ax, (name, corr) in zip(axes.flat, [('R&D', df_RnD_corr), ('BB1', df_BB1_corr)]):
        
        cax = ax.matshow(corr,cmap='coolwarm', vmin=-1, vmax=1)

        #Ejes
        ticks = np.arange(0,len(corr.columns),1)
        ax.set_xticks(ticks)
        ax.set_yticks(ticks)
        ax.set_xticklabels(corr.columns, rotation=90)
        ax.set_yticklabels(corr.columns)
        ax.xaxis.tick_top()
        ax.set_title('{}'.format(name), fontsize=16)
        
     
    # Mapa de color
    fig.colorbar(cax, ax=axes.ravel().tolist(), fraction=0.025, pad=0.04)
    # Título
    #fig.suptitle('Mapa de correlación de {}'.format(label), fontsize=25)
    
    #plt.show()
    plt.savefig('./../../figuras/datospp-correlaciones-{}'.format(label), bbox_inches='tight', facecolor=fig.get_facecolor(),edgecolor='none')
    plt.close()

```{figure} ./../../figuras/datospp-correlaciones-fondo.png
---
name: datospp-correlaciones-fondo
width: 80%
---
Mapa de correlaciones para los eventos de **fondo**.
```
```{figure} ./../../figuras/datospp-correlaciones-señal.png
---
name: datospp-correlaciones-señal
width: 80%
---
Mapa de correlaciones para los eventos de **señal**.
```

La mayoría de las variables no están correlacionadas. Sin embargo, tanto para la señal como para el fondo, el $p_T$ de los dos jets principales y $\phi$ de ambos jets están correlacionados. Para la señal, los $p_T$ de los jets principales presentan correlaciones negativas con $\Delta R$.

In [34]:
variables = [('pT_j1', 'pT_j2'), ('phi_j1', 'phi_j2'), ('pT_j1', 'deltaR_j12'), ('pT_j2', 'deltaR_j12')]
lista_RnD = create_png('RnD', df_RnD, variables, 'corr', type='scatter')
lista_BB1 = create_png('BB1', df_BB1, variables, 'corr', type='scatter')

# Grid de imagenes para RnD y BB1
image_grid(rows=2, columns=2, images=lista_RnD, title='datospp-correlaciones-RnD')
image_grid(rows=2, columns=2, images=lista_BB1, title='datospp-correlaciones-BB1')

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

```{figure} ./../../figuras/datospp-correlaciones-RnD.png
---
name: datospp-correlaciones-fondo
width: 100%
---
Variables correlacionadas del conjunto R&D.
```

```{figure} ./../../figuras/datospp-correlaciones-BB1.png
---
name: datospp-correlaciones-fondo
width: 100%
---
Variables correlacionadas del conjunto BB1.
```
La correlación del $p_T$ de los jets es lineal por la naturaleza del evento y del agrupamiento. Se escogieron como jets principales los más energéticos, por lo que el $p_T$ de los jets secundarios puede ser a lo sumo igual que el de los jets principales. Además, el momento de la colisión se distribuye de acuerdo a los dos productos principales que se formen, manteniendo una relación entre sí. 

Los valores de $\phi$ también presentan una correlación positiva por la naturaleza del evento. En los eventos dijet, los jets se generan en direcciones opuestas, por lo que la dirección angular del jet principal se relaciona con la del jet secundario. Las demás correlaciones no son evidentes en los gráficos.

La correlación de los datos puede generar problemas al utilizar algoritmos de aprendizaje automático para regresión. Sin embargo, debido a los modelos que se van a utilizar y a que la tarea es de clasificación, las consideraciones respecto a la correlación de las variables relacionadas al entrenamiento de los modelos no son necesarias.

Para la señal del conjunto BB1 parece haber una correlación de $\tau_{21}\_j2$ y $E\_j2$ con el $p_T$ del jet secundario. Sin embargo, por la poca cantidad de señal, no es evidente en los gráficos.

In [117]:
lista_BB1 = create_png('BB1', df_BB1, [('tau_21_j2','pT_j2'),('E_j2', 'pT_j2')], 'correlaciones-senal', type='scatter')

# Grid de imagenes para RnD y BB1
image_grid(rows=1, columns=2, images=lista_BB1, title='datospp-correlaciones-senal-BB1')

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

```{figure} ./../../figuras/datospp-correlaciones-senal-BB1.png
---
name: datospp-correlaciones-senal-BB1
width: 100%
---
Correlaciones de la señal del conjunto BB1 que no se encuentran en el conjunto R&D.
```